# 0. Meta

## 0.1. Packages

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

## 0.2. Functions

# 1. Data import

The data originates from Kaggle, specifically the dataset titled ["Cars Germany"](https://www.kaggle.com/datasets/ander289386/cars-germany/data). Spanning from 2011 to 2021, the dataset encompasses information sourced from AutoScout24, one of Europe's major car markets for new and used vehicles. The data collection and scraping were facilitated by an automated tool developed at ZenRows, capturing essential details such as make, model, mileage, horsepower, and more.

In [8]:
df_raw = pd.read_csv("../data/raw/autoscout24.csv")

# 2. Initial Data exploration

## 2.1. Structure of Data

## 2.2. Scale of Measure

## 2.2. Missing Entries

## 2.3. Categorial Variables

### 2.3.1. Counts per categorical level

### 2.3.2. Target distribution by categorical levels

## 2.4. Continuous Variables

### 2.4.1. Correlation

### 2.4.2. Linearity

### 2.4.3. Distribution

### 2.4.4. Outliers